<a href="https://colab.research.google.com/github/nguyenanhtienabcd/AIO2024_EXERCISE/blob/feature%2FMODULE7-WEEK3/m07w03_ex1_optimized_version.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Optimized version

### Install library

In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 914.6/914.6 kB 53.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 109.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 99.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [ ]:
!pip install loguru

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 6.2 MB/s eta 0:00:00


### Program

In [ ]:
# import các thư viện cần thiết
import argparse
from collections import defaultdict
import cv2
import numpy as np
from tqdm import tqdm
from ultralytics import YOLO
from loguru import logger

In [ ]:
# định nghĩa cấu hình khởi tạo video
def load_config():
  """load và trả lại cấu hình"""
  return {
      "model_path": "yolo11x.pt",
      "track_history_length": 120,
      "batch_size": 64,
      "line_thickness": 4,
      "track_color": (230, 230, 230),  # Add track_color (green in this case)
  }

In [ ]:
def initialize_video(video_path):
  """Initialize video capture and writer object"""
  cap =  cv2.VideoCapture(video_path)
  width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
  height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
  fps = int(cap.get(cv2.CAP_PROP_FPS))

  video_name = video_path.split("/")[-1].split(".")[0]
  output_path = f"/content/{video_name}_tracked.mp4"
  fourcc = cv2.VideoWriter_fourcc(*"mp4v")
  out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
  return cap, out, output_path

In [ ]:
def update_track_history(
    track_history,
    last_seen,
    track_ids,
    frame_count,
    batch_size,
    frame_idx,
    history_length,
):
  """Update track history and remove old tracks"""
  current_track = set(track_ids) # lấy toàn bộ track_id trong
  for track_id in list(track_history.keys()):
    if track_id in current_track:
      last_seen[track_id] = frame_count - (batch_size - frame_idx - 1)
    elif frame_count - last_seen[track_id] > history_length:
      del track_history[track_id]
      del last_seen[track_id]



In [ ]:
def draw_tracks(frame, boxes, track_ids, track_history, config):
  """Draw tracking lines on frame"""
  if not track_ids:
    return frame

  for box, track_id in zip(boxes, track_ids):
    x, y, h, w = box  # lấy tọa độ x, y
    track = track_history[track_id]  # lấy id
    track.append((float(x), float(y)))  # lưu tọa độ vaod ID
    if len(track) > config["track_history_length"]:  # kiểm tra track_his
      track.pop(0)  # Use pop(0) to remove the first element

    points = np.hstack(track).astype(np.int32).reshape((-1, 1, 2))
    cv2.polylines(
        frame,
        [points],
        isClosed=False,  # Add isClosed argument
        color=config["track_color"],
        thickness=config["line_thickness"],
    )  # dùng để vẽ hình có chứa tọa độ
  return frame # Move return statement outside the loop

In [ ]:
""" Xử lý một patch các frame """
def process_batch(model, batch_frames, track_history, last_seen, frame_count, config):
    """process batch of frames through YOLO model"""
    results = model.track(
        batch_frames,
        persist=True,
        verbose=False,
        show=False,
        tracker="botsort.yaml",
        iou = 0.5,
    )
    processed_frames = []
    for frame_idx, result in enumerate(results):
        # Now 'result' is the result object for a single frame

        # Access boxes and track_ids from the individual result object
        boxes = result.boxes.xywh.cpu()
        # Get track_ids, ensure it's a list
        track_ids = (
            result.boxes.id.int().cpu().tolist() if result.boxes.id is not None else []
        )

        update_track_history(
            track_history,
            last_seen,
            track_ids,
            frame_count,
            len(batch_frames),
            frame_idx,
            config["track_history_length"],
        )

        annotated_frame = result.plot(font_size=4, line_width=2)
        annotated_frame = draw_tracks(
            annotated_frame,
            boxes,
            track_ids,
            track_history,
            config,
        )
        processed_frames.append(annotated_frame)

    # Return the list of processed frames after processing the entire batch
    return processed_frames

In [ ]:
def main(video_path):
  """ Main function to process video"""
  CONFIG = load_config() # load toàn bộ cấu hình mong muốn
  model = YOLO(CONFIG.get("model_path", "yolo11x.pt")) # lấy model mong muốn
  cap, out, output_path = initialize_video(video_path) # lấy khung hình, video đầu ra, đương lưu video đầu ra
  track_history = defaultdict(lambda: []) # tạo ra một track theo dõi
  last_seen = defaultdict(int) # tạo ta một list
  total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) # lấy toàn bộ frame của video

  with tqdm(total=total_frames,
            desc = "Processing frames",
            colour = "green",) as pbar:
            frame_count = 0
            batch_frame = []

            while cap.isOpened():
              sucess, frame = cap.read()
              if not sucess:
                break

              frame_count += 1
              batch_frame.append(frame)

              if len(batch_frame) == CONFIG["batch_size"] or frame_count == total_frames:
                  try:
                      processed_frames = process_batch(
                          model,
                          batch_frame,
                          track_history,
                          last_seen,
                          frame_count,
                          CONFIG,
                      )
                      for frame in processed_frames:
                        out.write(frame)
                        pbar.update(1)
                      batch_frame = []
                  except Exception as e:
                      logger.error(f"Error processing batch: {e}")
                      batch_frame = []
                      continue
  try:
    cap.release()
    out.release()
    cv2.destroyAllWindows()
  except Exception as e:
    logger.error(f"Error releasing resources: {e}")


In [ ]:
def main(video_path):
    """ Main function to process video"""
    CONFIG = load_config()  # load toàn bộ cấu hình mong muốn
    model = YOLO(CONFIG.get("model_path", "yolo11x.pt"))  # lấy model mong muốn
    cap, out, output_path = initialize_video(video_path)  # lấy khung hình, video đầu ra, đường dẫn video đầu ra
    track_history = defaultdict(lambda: [])
    last_seen = defaultdict(int)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    with tqdm(total=total_frames, desc="Processing frames", colour="green") as pbar:
        frame_count = 0
        batch_frame = []

        while cap.isOpened():
            success, frame = cap.read()
            if not success:
                break

            frame_count += 1
            batch_frame.append(frame)

            # Xử lý batch khi đủ kích thước hoặc đã đọc tới frame cuối (theo số frame báo của video)
            if len(batch_frame) == CONFIG["batch_size"] or frame_count == total_frames:
                try:
                    processed_frames = process_batch(
                        model,
                        batch_frame,
                        track_history,
                        last_seen,
                        frame_count,
                        CONFIG,
                    )
                    for processed_frame in processed_frames:
                        out.write(processed_frame)
                        pbar.update(1)
                    batch_frame = []  # reset batch sau khi xử lý
                except Exception as e:
                    logger.error(f"Error processing batch: {e}")
                    batch_frame = []
                    continue

        # Sau vòng lặp, nếu còn frame chưa được xử lý (batch cuối không đủ số lượng), tiến hành xử lý
        if batch_frame:
            try:
                processed_frames = process_batch(
                    model,
                    batch_frame,
                    track_history,
                    last_seen,
                    frame_count,
                    CONFIG,
                )
                for processed_frame in processed_frames:
                    out.write(processed_frame)
                    pbar.update(1)
            except Exception as e:
                logger.error(f"Error processing final batch: {e}")

    try:
        cap.release()
        out.release()
        cv2.destroyAllWindows()
    except Exception as e:
        logger.error(f"Error releasing resources: {e}")


**Lỗi xử lý batch cuối**: Code xử lý video theo batch. Khi số lượng khung hình còn lại không đủ tạo thành một batch, đoạn code có thể không xử lý được batch cuối cùng này

In [ ]:
video_path = "/content/vtv24h.mp4"
main(video_path)

Processing frames:  99%|█████████▊| 313/317 [00:27<00:00, 11.49it/s]
